In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from matplotlib import cm
from tqdm import tqdm
import json

import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc
import squidpy as sq
from scipy.spatial import KDTree
import networkx as nx


from src.slide_utilities import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read metadata: resolution_dict
with open("res_indices.json", 'r') as file:
    resolution_dict = json.load(file)
sample_set = resolution_dict.keys()

In [3]:
adata = sc.read_h5ad("/data/projects/Behnam/xenium-cgn/data/Glom_NichePCA_Ann_all_with_clusters_6.h5ad")

## Plot numbers for gloms

In [62]:
# sample_set = [1]

for sample in sample_set:

    del adata_i
    adata_i = adata[adata.obs['Sample_num'] == f'X{sample}']
    
    dpi = 100
    plt.figure(dpi=dpi)
    
    color = [1 if i == 'glom' else 0 for i in adata_i.obs['nichepca_ann']]
    
    plt.scatter(adata_i.obs['x'], adata_i.obs['y'], s=.01, c=color, cmap='viridis') # other colormaps: viridis, plasma, inferno, magma, cividis
    plt.gca().set_aspect('equal', adjustable='box')
    plt.gca().invert_yaxis() 
    plt.gca().set_xticks([])
    plt.gca().set_yticks([]) 

    glom_no = adata_i.obs['nichepca_glom_no'].unique()
    for i in glom_no:
        if i != 'Other':
            centroid = adata_i.obs.loc[adata_i.obs['nichepca_glom_no'] == i, ['x', 'y']].mean()
            plt.text(centroid[0], centroid[1], f"{i}", fontsize=8, ha='center', va='center', c='red')
    
    plt.savefig(f"glom_num/sample_X{sample}.pdf", format='pdf', bbox_inches='tight')
    # plt.show()
    plt.close() 

## Plot only gloms in jpg format

In [6]:
window = 200
dpi = 600

for sample in tqdm(sample_set):

    if int(sample) < 44:
        continue
        
    adata_i = adata[adata.obs['Sample_num'] == f'X{sample}']
    
    glom_no = adata_i.obs['nichepca_glom_no'].unique()
    for i in glom_no:
        if i != 'Other':
            x, y = adata_i.obs.loc[adata_i.obs['nichepca_glom_no'] == i, ['x', 'y']].mean()

            x_min = x - window
            x_max = x + window
            y_min = y - window
            y_max = y + window

            fig, ax = plt.subplots(figsize=(5, 5))
            sc.pl.spatial(adata_i[(adata_i.obs['x'] > x_min) & (adata_i.obs['x'] < x_max) & (adata_i.obs['y'] > y_min) & (adata_i.obs['y'] < y_max)],
                          library_id=adata_i.obs['Slide_ID'][0],
                          img_key='he_downscaled_aligned',
                          ax=ax,
                          show=False)
            
            # Set aspext ratio, remove the box and text
            plt.gca().set_aspect('equal', adjustable='box')
            plt.xlabel('')
            plt.ylabel('')
            for spine in plt.gca().spines.values():
                spine.set_visible(False)
            
            plt.savefig(f"glom_single_dpi600/sample_X{sample}_glom_{i}.jpg", format='jpg', bbox_inches='tight', dpi=dpi)
            # plt.show
            plt.close(fig)
            
    del adata_i
    

100%|██████████| 60/60 [08:09<00:00,  8.16s/it]
